# 定点乘法

测试浮点数到定点的转换。

构造具有大范围浮点值的 numpy 数组来实现。使用 {func}`tvm.topi.hexagon.utils.get_fixed_point_value` 将这些值转换为定点值。接着，使用函数提供的 `scale_factor` 将这些值转换回浮点数。然后将这些转换后的浮点值与原始值进行比较，如果它们恰好超出了预期的容差，就会触发断言。

In [3]:
import numpy as np
import struct
import math
import tvm
from tvm.topi.hexagon.utils import get_fixed_point_value

In [5]:
# 构造具有广泛值范围的数组
fp1 = np.random.uniform(0.00001, 0.0002, size=(10))
fp2 = np.random.uniform(0.001, 0.02, size=(10))
fp3 = np.random.uniform(1, 20, size=(10))
fp4 = np.random.uniform(900, 1000, size=(10))
fp5 = np.random.uniform(1e9, 1e10, size=(10))

# 根据 IEEE-754 浮点标准测试具有最大可能指数的值(实际 exp 值 = 127，存储 exp 值 = 254)
fp6 = np.random.uniform(2.4e38, 2.5e38, size=(1))
# 测试非常小的浮点值
fp7 = np.random.uniform(1.4e-34, 1.7e-34, size=(1))

float_arr = np.concatenate((fp1, fp2, fp3, fp4, fp5, fp6, fp7))
for flp in float_arr:
    fxp, rsh = get_fixed_point_value(flp, "int16")
    # 使用 rsh 计算 scale_factor (rsh 是 scale_factor 的log2)。
    # 这样做的时候，使用IEEE-754浮点表示，因为 rsh 可以是负数或正数。

    scale = ((rsh + 127) & 0xFF) << 23 # 添加偏置 (127) 并将其定位到指数位
    scale_i = struct.pack("I", scale)  # Pack 作为整数
    scale_f = struct.unpack("f", scale_i)  # Unpack 作为浮点数

    converted_flp = fxp / scale_f[0]
    assert math.isclose(flp, converted_flp, rel_tol=1e-2)